# Setup 

### Importing 

In [4]:
# imports functions from the other notebooks
# requires nbimporter
import sys, os
sys.path.append(os.path.join(os.path.dirname(''), '..'))
#sys.path.append(os.path.join(os.path.dirname(''), '../..'))
from proj3_gans_scooters.src.scraping import scrape_wiki_cities, scrape_weather, icao_airport_codes
from proj3_gans_scooters.src.utils import PrivateKeysHandler, MyMySQLConnection

### Load Private Keys

In [5]:
# requires a file '.env' at the same level at the main file with :
#      [APIs]
#      openweather_key = <key1>
#      aerodatabox_key = <key2>
# where <key> are the keys without quotes or anything

relative_path_to_file = '.env'
keys = PrivateKeysHandler(relative_path_to_file)
api_key_dict = keys.load_keys('APIs')
sql_cred_dict = keys.load_keys('SQL')

### Create MySql Connection

In [6]:
cnx = MyMySQLConnection(sql_cred_dict)

# Scraping
* cities
* weather
* flight data

In [7]:
cities_df = scrape_wiki_cities()

In [8]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   city                94 non-null     object        
 1   member_state        94 non-null     object        
 2   officialpopulation  94 non-null     int32         
 3   date                94 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 2.7+ KB


In [9]:
weather_df = scrape_weather(list(cities_df.city), api_key_dict['openweather_key'])

In [10]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3760 entries, 0 to 3759
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   city                     3760 non-null   object        
 1   date                     3760 non-null   datetime64[ns]
 2   temp_celcius             3760 non-null   float64       
 3   temp_feels_like_celcius  3760 non-null   float64       
 4   humidity_percent         3760 non-null   int64         
 5   weather_description      3760 non-null   object        
 6   visibility               3760 non-null   int64         
 7   wind_speed_meter_sec     3760 non-null   float64       
 8   wind_direction_degree    3760 non-null   int64         
 9   wind_gust_meter_sec      3760 non-null   float64       
 10  pop_percent              3760 non-null   float64       
 11  rain_3h_mm               3760 non-null   float64       
 12  pod                      3760 non-

In [8]:
icao_airports_df = icao_airport_codes(latitudes, longitudes, api_key_dict['aerodatabox_key'])

NameError: name 'latitudes' is not defined

# Adding to Database

In [23]:
import sys, os
sys.path.append(os.path.join(os.path.dirname(''), '../..'))
from proj3_gans_scooters.src.utils import install_pip_pkg
    
class yMySQLConnection:
    from pandas import DataFrame
    def __init__(
            self, 
            credentials : dict, 
            db_name : str = 'gans_scooters',
            host : str = "127.0.0.1",
            port : int = 3306):
                
        user = credentials['user']
        password = credentials['password']
        self.db_name = db_name
        
        # setup SQLAlchemy   
        from sqlalchemy import create_engine 
        install_pip_pkg({'pymysql'})
        
        dialect = 'mysql'
        driver = 'pymysql'
        cnx = f'{dialect}+{driver}://{user}:{password}@{host}:{port}/' #{db_name}'
        
        # create database if not already created
        self.alch_engine = create_engine(
            cnx, 
            connect_args={'connect_timeout': 10}, 
            echo=False
        ) 
        with self.alch_engine.begin() as con:
            cursor = con.execute("SHOW DATABASES")
            if (self.db_name,) not in list(cursor):
                con.execute(f"CREATE DATABASE {self.db_name}")
            con.execute(f"USE {self.db_name}")
                
    # tables = {table1 : ([columns], [args per column]),
    #           table2 : ([....
    # }
    # i.e
    # tables = {'Customers' : (['ID', 'Name',...], ['int NOT NULL AUTO_INCREMENT', 'varchar(255) NOT NULL']),
    #   ...}
    def create_tables(self, tables : dict):        
        with self.alch_engine.begin() as con:
            for table, (columns, args) in tables.items():
                con.exec_driver_sql(f"DROP TABLE IF EXISTS {table}")
                cols = ',\n'.join(map(lambda x: x[0] + ' ' + x[1], zip(columns, args)))
                con.execute(f"CREATE TABLE {table} ({cols});")
                 
    def execute(self, query : str):
        with self.alch_engine.begin() as con:
            return con.execute(query)
        
    def add_tables_to_db(
            self, 
            dfs : list, 
            tablenames : list, 
            insert_modes : list,
            id_columns : list = None
            ):
        
        if not id_columns: # only goes in if None
            id_columns = [None] * len(dfs)
            
        assert(len(dfs) == len(tablenames))
        assert(len(dfs) == len(id_columns))
        assert(len(dfs) == len(insert_modes))
        
        # 'begin' opens a transaction and the 'with' environment cares for a rollback if something 
        # goes wrong
        with self.alch_engine.begin() as con:
            for i, df in enumerate(dfs):

                # replace messes up the column types
                # instead emptying the table and then appending
                # keeps all the constraints
                if insert_modes[i] == 'replace':
                    con.execute(f"TRUNCATE TABLE {tablenames[i]}")
                    insert_modes[i] = 'append'

                df.to_sql(
                    name= tablenames[i],
                    if_exists=insert_modes[i], #'append', #'replace'
                    con=con,
                    schema=self.db_name,
                    index=False,
                    #chunksize=1000
                )

                #if id_columns[i]: #only enter if not None
                #    con.execute(f'ALTER TABLE `{tablenames[i]}` ADD PRIMARY KEY (`{id_columns[i]}`);') 
    
    
    
    def add_weather_to_db(
            self, 
            df : DataFrame, 
            insert_mode : str):
                
        with self.alch_engine.begin() as con:
            cities = pd.read_sql('SELECT city_id, city FROM cities;', con)
            df.assign(city_id = cities.loc[cities['city']==df['city'], 'city_id'])
        

                
    def add_table_to_db(
            self, 
            df : DataFrame, 
            tablename : str, 
            insert_mode : str,
            id_column : str):
        
        with self.alch_engine.begin() as con:
            df.to_sql(
                #schema="dbo"
                name= tablename,
                if_exists=insert_mode, #'replace'
                con=con, 
                schema=self.db_name,
                index=False,
                chunksize=1000
            )
    
            #con.execute(f'ALTER TABLE `{tablename}` ADD PRIMARY KEY (`{id_column}`);')
              #not null, unique, default, primary key, foreign key

In [24]:
cnx.execute('drop database gans_scooters')
del cnx

In [25]:
cnx = yMySQLConnection(sql_cred_dict)
cities_primary = 'city_id'
cities_cols = [cities_primary]
cities_cols.extend(cities_df.columns)

weather_cols = [cities_primary]
weather_cols.extend(weather_df.columns[1:-1])
weather_cols.append('snow_3h_mm')
weather_foreign_key = f'FOREIGN KEY ({cities_primary}) REFERENCES cities({cities_primary})'

d = {'cities': (cities_cols, [f'int NOT NULL AUTO_INCREMENT, PRIMARY KEY ({cities_primary})', #city_id
                              'varchar(255) NOT NULL',       #city 
                              'varchar(255)',                #member_state
                              'int',                         #officialpopulation
                              'DATETIME']),                   #date
     'weather': (weather_cols, [f'int NOT NULL, {weather_foreign_key}',#city_id
                                'DATETIME NOT NULL',        #date
                                'DECIMAL(4, 2)',             #temp_celcius
                                'DECIMAL(4, 2)',             #temp_feels_like_celcius                                 
                                'INT(3)',                    #humidity_percent                                     
                                'varchar(255)',              #weather_description                                       
                                'INT(5)',                    #visibility     
                                'DECIMAL(5, 2)',             #wind_speed_meter_sec                                     
                                'INT(3)',                    #wind_direction_degree       
                                'DECIMAL(5, 2)',             #wind_gust_meter_sec     
                                'DECIMAL(5, 2)',             #pop_percent       
                                'DECIMAL(5, 2) DEFAULT 0.00',#rain_3h_mm        
                                'DECIMAL(5, 2) DEFAULT 0.00']), #snow_3h_mm   
    }

cnx.create_tables(d)

In [26]:
res = cnx.add_table_to_db(
            df = cities_df,
            tablename = 'cities',
            insert_mode = 'append',
            id_column = 'city_id'
)
print(res)

None


In [ ]:
pd.read_sql('SELECT city_id, city FROM cities;', 

In [28]:
tmp = cnx.execute(f'SELECT city_id, city FROM cities;')
for i in tmp:
    print(i)

(1, 'Berlin')
(2, 'Madrid')
(3, 'Rome')
(4, 'Bucharest')
(5, 'Paris')
(6, 'Vienna')
(7, 'Hamburg')
(8, 'Warsaw')
(9, 'Budapest')
(10, 'Barcelona')
(11, 'Munich')
(12, 'Milan')
(13, 'Sofia')
(14, 'Prague')
(15, 'Cologne')
(16, 'Stockholm')
(17, 'Naples')
(18, 'Amsterdam')
(19, 'Marseille')
(20, 'Turin')
(21, 'Valencia')
(22, 'Kraków')
(23, 'Zagreb')
(24, 'Frankfurt am Main')
(25, 'Seville')
(26, 'Zaragoza')
(27, 'Łódź')
(28, 'Athens')
(29, 'Helsinki')
(30, 'Rotterdam')
(31, 'Wrocław')
(32, 'Copenhagen')
(33, 'Palermo')
(34, 'Stuttgart')
(35, 'Düsseldorf')
(36, 'Riga')
(37, 'Leipzig')
(38, 'Dortmund')
(39, 'Gothenburg')
(40, 'Essen')
(41, 'Málaga')
(42, 'Bremen')
(43, 'Genoa')
(44, 'Vilnius')
(45, 'Dresden')
(46, 'Dublin')
(47, 'The Hague')
(48, 'Hanover')
(49, 'Poznań')
(50, 'Antwerp')
(51, 'Lyon')
(52, 'Nuremberg')
(53, 'Lisbon')
(54, 'Duisburg')
(55, 'Toulouse')
(56, 'Bratislava')
(57, 'Gdańsk')
(58, 'Murcia')
(59, 'Tallinn')
(60, 'Palma de Mallorca')
(61, 'Szczecin')
(62, 'Sintra')
(

In [17]:
tmp_weather_df = weather_df[['date', 'temp_celcius', 'temp_feels_like_celcius',
       'humidity_percent', 'weather_description',
       'wind_speed_meter_sec', 'wind_direction_degree', 'wind_gust_meter_sec',
       'pop_percent', 'rain_3h_mm']]
cnx.add_tables_to_db(
            [cities_df, tmp_weather_df], 
            list(d.keys()), 
            ['append', 'append']
)

OperationalError: (pymysql.err.OperationalError) (1364, "Field 'city_id' doesn't have a default value")
[SQL: INSERT INTO gans_scooters.weather (date, temp_celcius, temp_feels_like_celcius, humidity_percent, weather_description, wind_speed_meter_sec, wind_direction_degree, wind_gust_meter_sec, pop_percent, rain_3h_mm) VALUES (%(date)s, %(temp_celcius)s, %(temp_feels_like_celcius)s, %(humidity_percent)s, %(weather_description)s, %(wind_speed_meter_sec)s, %(wind_direction_degree)s, %(wind_gust_meter_sec)s, %(pop_percent)s, %(rain_3h_mm)s)]
[parameters: ({'date': datetime.datetime(2022, 6, 10, 12, 0), 'temp_celcius': 21.88, 'temp_feels_like_celcius': 21.3, 'humidity_percent': 45, 'weather_description': 'few clouds', 'wind_speed_meter_sec': 3.83, 'wind_direction_degree': 290, 'wind_gust_meter_sec': 5.34, 'pop_percent': 0.0, 'rain_3h_mm': 0.0}, {'date': datetime.datetime(2022, 6, 10, 15, 0), 'temp_celcius': 23.64, 'temp_feels_like_celcius': 23.03, 'humidity_percent': 37, 'weather_descriptio

In [13]:
#!pip show pandas

Name: pandas
Version: 1.4.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: c:\users\circl\anaconda3\lib\site-packages
Requires: numpy, python-dateutil, pytz
Required-by: statsmodels, seaborn


Das System kann den angegebenen Pfad nicht finden.


In [15]:
tmp = cnx.execute('DESCRIBE cities;')
for i in tmp:
    print(i)
tmp = cnx.execute('DESCRIBE weather;')
for i in tmp:
    print(i)

('city_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('city', 'varchar(255)', 'NO', '', None, '')
('member_state', 'varchar(255)', 'YES', '', None, '')
('officialpopulation', 'int', 'YES', '', None, '')
('date', 'datetime', 'YES', '', None, '')
('city_id', 'int', 'NO', 'MUL', None, '')
('date', 'datetime', 'NO', '', None, '')
('temp_celcius', 'decimal(4,2)', 'YES', '', None, '')
('temp_feels_like_celcius', 'decimal(4,2)', 'YES', '', None, '')
('humidity_percent', 'int', 'YES', '', None, '')
('weather_description', 'varchar(255)', 'YES', '', None, '')
('visibility', 'int', 'YES', '', None, '')
('wind_speed_meter_sec', 'decimal(5,2)', 'YES', '', None, '')
('wind_direction_degree', 'int', 'YES', '', None, '')
('wind_gust_meter_sec', 'decimal(5,2)', 'YES', '', None, '')
('pop_percent', 'decimal(5,2)', 'YES', '', None, '')
('rain_3h_mm', 'decimal(5,2)', 'YES', '', '0.00', '')
('snow_3h_mm', 'decimal(5,2)', 'YES', '', '0.00', '')


In [16]:
print(cities_df.columns, weather_df.columns)

Index(['city', 'member_state', 'officialpopulation', 'date'], dtype='object') Index(['city', 'date', 'temp_celcius', 'temp_feels_like_celcius',
       'humidity_percent', 'weather_description', 'visibility',
       'wind_speed_meter_sec', 'wind_direction_degree', 'wind_gust_meter_sec',
       'pop_percent', 'rain_3h_mm', 'pod'],
      dtype='object')


In [35]:
db_name = 'gans_scooters'
host ="127.0.0.1"
port = 3306
user = sql_cred_dict['user']
password = sql_cred_dict['password']
dialect = 'mysql'
driver = 'pymysql'
cnx = f'{dialect}+{driver}://{user}:{password}@{host}:{port}/' #{db_name}'

import sqlalchemy as sa
import pandas as pd
# create database if not already created
alch_engine = sa.create_engine(
    cnx, 
    connect_args={'connect_timeout': 10}, 
    echo=False
) 
with alch_engine.begin() as con:
    cursor = con.execute("SHOW DATABASES")
    if (db_name,) not in list(cursor):
        con.execute(f"CREATE DATABASE {db_name}")
    con.execute(f"USE {db_name}")
    
    cities = pd.read_sql('SELECT city_id, city FROM cities;', con)
    df.assign(city_id = cities.loc[cities['city']==df['city'], 'city_id'])

NameError: name 'df' is not defined